### Build a Crew to Tailor Job Applications

In [22]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display
from crewai_tools import FileReadTool, ScrapeWebsiteTool, MDXSearchTool, SerperDevTool

In [3]:
#---- Load environment and model -------

load_dotenv()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-mini'

In [4]:
# ----- Tools ------

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./MyFakeResume.md')
semantic_search_resume = MDXSearchTool(mdx='./MyFakeResume.md')

In [5]:
# from IPython.display import Markdown, display
# display(Markdown("./MyFakeResume.md"))

### Creating Agents

In [6]:
#------ Agent1: Researcher ------

researcher = Agent(
    role="Tech Job Researcher",
    goal="Exract key skills and requirements from job postings.",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "You are an expert at analyzing technical job postings with exceptional skill."
        "You can extract accurate information and hidden requirements from job descriptions."
        "Your analyses help accurately identify the required technical and personal skills."
        "Your skills help pinpoint the necessary qualifications and skills sought "
        "by employers, forming the foundation for effective application tailoring."
    )
)

In [7]:
#------- Agent2: Profiler --------

profiler = Agent(
    role= "Personal Profiler Analyst",
    goal= "Analysis of resumes, multiple data sources and personal projects to identify "
          "strengths and distinctions that match job requirements to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
          "You have exceptional analytical skills for evaluating developer profiles, "
          "and synthesize insights from multiple data sources to create rich candidate "
          "profiles that reflects strengths, achievements, and personality."
    )
)

In [8]:
#------- Agent3: Resume Strategist ----------

resume_strategist = Agent(
    role="Resume Strategist",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [9]:
#------- Agent4: Cover Letter Writer -------

cover_letter_writer = Agent(
    role = "Cover Letter Writer",
    goal = "Write compelling and customized cover letters.",
    verbose=True,
    backstory = "You transform candidate data into convincing narratives that connect "
                "skills and experiences with job requirements."
)

In [10]:
#------- Agent5: Interview Preparer -------

interview_preparer = Agent(
    role= " Interview Coach",
    goal= "Create interview questions and talking points "
          "based on the resume and job description.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

### Creating Tasks

In [11]:
# ------ Task1: Extract Job Requirements ----

research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)  

In [12]:
#------ Task2: Compile Comprehensive Profile ------

profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [13]:
#------ Task3: Align Resume with Job Requirements ------

resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file= "tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [14]:
#------ Task4: Cover Letter Task ------

cover_letter_task = Task(
    description = (
        "Using the candidate’s tailored resume, profile and job requirements obtained from "
        "previous tasks, write a tailored cover letter."
        "The cover letter should highlight "
        "the most relevant skills, experiences, and achievements from the candidate’s "
        "background, and clearly demonstrate how they meet the employer’s needs."
        "Ensure the narrative is professional, compelling, and customized for the "
        "given role without inventing information."       
    ),
    expected_output = (
        "A well-structured and polished cover letter (in Markdown format) that "
        "emphasizes the candidate’s alignment with the job posting, highlighting "
        "their strengths and making a convincing case for their candidacy."
    ),        
    output_file = "cover_letter.md",
    context = [research_task, profile_task, resume_strategy_task],
    agent = cover_letter_writer,
    
)

In [15]:
#------- Task5: Develop Interview Materials ---------

interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

### Creating the Crew

In [16]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            cover_letter_writer,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           cover_letter_task,
           interview_preparation_task],

    verbose=True
)

### Running the Crew

In [17]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/b081cf84-ca63-4475-a4f9-5037565d2804',
    'github_url': 'https://github.com/Rajaa-Rahil',
    'personal_writeup': """Hafsa Ali is a distinguished Data Scientist with over 7 years of 
     experience applying statistical modeling, machine learning, and data visualization to 
     solve complex business challenges. She holds a Master's degree in Computer Science, 
     has a strong background in AI and data science, and is an expert in multiple programming 
     languages and frameworks. Hafsa has led projects deploying scalable AI solutions and 
     innovative data models. She is an ideal candidate for senior data science and 
     executive-level roles requiring both technical expertise and strategic leadership.""" 
}

In [20]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 907fd1ef-8095-469f-865e-5923b466d352                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Job Researcher                                                                                     │
│                                                                                                                 │
│  Task: Analyze the job posting URL provided                                                                     │
│  (https://jobs.lever.co/AIFund/b081cf84-ca63-4475-a4f9-5037565d2804) to extract key skills, experiences, and    │
│  qualifications required. Use the tools to gather content and identify and categorize the requirements.         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Personal Profiler Analyst                                                                               │
│                                                                                                                 │
│  Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/Rajaa-Rahil)   │
│  URLs, and personal write-up (Hafsa Ali is a distinguished Data Scientist with over 7 years of                  │
│       experience applying statistical modeling, machine learning, and data visualization to                     │
│       solve complex business challenges. She holds a Master's degree in Computer Science,                       │
│       has a strong background in AI and data science, and is an expert in multiple programming                  │
│       languages and frameworks. Hafsa has led projects deploying scalable AI solutions and                      │
│       innovative data models. She is an ideal candidate for senior data science and                             │
│       executive-level roles requiring both technical expertise and strategic leadership.). Utilize tools to     │
│  extract and synthesize information from these sources.                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent:  Interview Coach                                                                                        │
│                                                                                                                 │
│  Task: Create a set of potential interview questions and talking points based on the tailored resume and job    │
│  requirements. Utilize tools to generate relevant questions and discussion points. Make sure to use these       │
│  question and talking points to help the candiadte highlight the main points of the resume and how it matches   │
│  the job posting.                                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent:  Interview Coach                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Interview Questions and Talking Points for Hafsa Ali**                                                       │
│  *Applying for Machine Learning Engineer (LatAm), RealAvatar at AI Fund*                                        │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 1. Experience with Multimodal AI Platforms and Pipelines                                                   │
│                                                                                                                 │
│  **Question:**                                                                                                  │
│  - Can you describe your experience building and optimizing multimodal AI pipelines involving audio, video,     │
│  and text data?                                                                                                 │
│  - How have you ensured low latency and streaming capabilities in real-time AI pipeline deployments?            │
│                                                                                                                 │
│  **Talking Points:**                                                                                            │
│  - Highlight your leadership in designing Python-based multimodal data pipelines that integrate streaming and   │
│  batch data (AI Horizons Consulting).                                                                           │
│  - Emphasize your work constructing real-time ETL pipelines optimized for low latency, supporting financial     │
│  risk assessment systems.                                                                                       │
│  - Discuss technical approaches for synchronizing and preprocessing multimodal datasets efficiently.            │
│  - Illustrate your knowledge of working with vector databases like Pinecone for similarity search in            │
│  multimodal contexts.                                                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 2. Machine Learning Model Development, Optimization & Deployment                                           │
│                                                                                                                 │
│  **Question:**                                                                                                  │
│  - What kinds of ML models have you developed and deployed, especially in environments requiring continuous     │
│  monitoring and optimization?                          

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 3f03ccd5-fb10-42ec-b290-4fee8d89e116                                                                     │
│  Agent:  Interview Coach                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 907fd1ef-8095-469f-865e-5923b466d352                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ---                                                                                              │
│                                                                                                                 │
│  **Interview Questions and Talking Points for Hafsa Ali**                                                       │
│  *Applying for Machine Learning Engineer (LatAm), RealAvatar at AI Fund*                                        │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 1. Experience with Multimodal AI Platforms and Pipelines                                                   │
│                                                                                                                 │
│  **Question:**                                                                                                  │
│  - Can you describe your experience building and optimizing multimodal AI pipelines involving audio, video,     │
│  and text data?                                                                                                 │
│  - How have you ensured low latency and streaming capabilities in real-time AI pipeline deployments?            │
│                                                                                                                 │
│  **Talking Points:**                                                                                            │
│  - Highlight your leadership in designing Python-based multimodal data pipelines that integrate streaming and   │
│  batch data (AI Horizons Consulting).                                                                           │
│  - Emphasize your work constructing real-time ETL pipelines optimized for low latency, supporting financial     │
│  risk assessment systems.                                                                                       │
│  - Discuss technical approaches for synchronizing and preprocessing multimodal datasets efficiently.            │
│  - Illustrate your knowledge of working with vector databases like Pinecone for similarity search in            │
│  multimodal contexts.                                                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 2. Machine Learning Model Development, Optimization & Deployment                                           │
│                                                                                                                 │
│  **Question:**                                                                                                  │
│  - What kinds of ML models have you developed and depl

#### Dislplay the generated tailored_resume.md file.

In [19]:
display(Markdown("./tailored_resume.md"))

```
Hafsa Ali  
San Francisco, CA | hafsa.ali@email.com | (123) 456-7890 | linkedin.com/in/hafsa-ali

PROFESSIONAL SUMMARY  
Machine Learning Engineer with 7+ years experience building scalable AI and data science solutions, specializing in multimodal data pipeline engineering, prototype machine learning model development, deployment, and monitoring. Proven expertise in designing and maintaining robust Python data pipelines for large-scale datasets in low latency streaming and batch environments. Skilled with cloud platforms including AWS, leveraging advanced ML frameworks and NLP techniques to deliver high-quality AI products. Adept at collaborating with cross-functional teams and stakeholders to drive innovative AI platform development in fast-paced startup settings. Passionate about automation, testing, and continuous model optimization to ensure performance and reliability.

PROFESSIONAL EXPERIENCE  

AI Horizons Consulting – Senior Data Scientist  
Remote | 2022 – Present  
- Architected and maintained scalable ETL and data pipelines for diverse multi-source financial and IoT datasets using Python and PostgreSQL, supporting real-time analytics and low latency scenarios.  
- Led development of prototype NLP and predictive models, focusing on regulatory document analysis and financial risk assessment, enhancing model evaluation and monitoring workflows.  
- Deployed ML models into production environments on AWS, implementing monitoring for model performance and concept drift.  
- Collaborated across AI, ML, and product teams to align data solutions with business goals, enabling effective decision-making in a dynamic environment.  
- Mentored data scientists on best practices in pipeline engineering, cloud deployment, and automated testing.

FinSight Analytics – Data Scientist  
New York, NY | 2019 – 2022  
- Developed and optimized classification models for credit-risk scoring, increasing predictive accuracy by 15%, incorporated in production pipelines.  
- Automated and maintained end-to-end data ingestion and preprocessing pipelines using Python and SQL, reducing reporting latency by 30%.  
- Worked closely with finance and engineering teams to integrate machine learning workflows and enhance data reliability on cloud infrastructure (AWS).  
- Delivered clear technical presentations and documentation to non-technical stakeholders, facilitating adoption of AI insights.

HealthData Solutions – Junior Data Scientist  
Boston, MA | 2016 – 2019  
- Built robust data cleaning and transformation scripts using Python and SQL for healthcare datasets, supporting predictive modeling for patient readmission risk.  
- Assisted in development and evaluation of ML models, contributing to improved patient care outcomes through data-driven insights.  
- Collaborated in team settings, demonstrating strong communication and attention to detail required for healthcare applications.

EDUCATION  
Master of Science in Data Science  
Florida Institute of Technology (FIT) – 2016  

Bachelor of Science in Computer Science  
University of Benghazi, Libya – 2014  

CERTIFICATIONS  
- AWS Certified Machine Learning – Specialty  
- TensorFlow Developer Certificate  
- Data Science Specialization – Johns Hopkins University (Coursera)  
- Tableau Desktop Specialist  

TECHNICAL SKILLS  
- Programming: Python (pandas, numpy, scikit-learn, TensorFlow, PyTorch), SQL/PostgreSQL, Bash  
- Data Engineering: ETL pipeline design, data ingestion, transformation, cleaning, workflow automation  
- ML & AI: Model prototyping, evaluation, deployment, monitoring, NLP (regulatory documents, social media sentiment)  
- Cloud Platforms: AWS (EC2, S3, SageMaker), familiar with GCP concepts  
- Databases: SQL, PostgreSQL; experience with handling vector data stores and large datasets  
- Tools/Libraries: Jupyter, Docker, Git, Pydantic, dotenv  
- Reporting & Visualization: matplotlib, seaborn, Tableau, automated Markdown/HTML reports  
- Collaboration: Agile methodologies, cross-functional teamwork, mentoring  

KEY PROJECTS  
- Financial NLP Pipeline: Developed large-scale natural language processing pipelines to extract insights from over 1 million regulatory documents, improving automations in compliance workflows.  
- Credit Scoring ML Pipeline: Engineered end-to-end classification model pipeline enhancing predictive accuracy and integrated with production data systems.  
- Healthcare Readmission Prediction: Built data preprocessing and ML model workflows streamlining hospital readmission risk analysis.  
- Multi-Agent AI Systems (Personal Projects): Designed frameworks coordinating intelligent agents for automation in event planning, procurement, and trading, showcasing expertise in multi-agent orchestration and LLM workflows.

PERSONAL ATTRIBUTES  
- Highly self-motivated and adaptable in startup and fast-moving tech environments.  
- Strong communicator, capable of explaining complex technical concepts clearly to diverse audiences.  
- Detail-oriented with a passion for automation, testing, and continuous improvement.  
- Collaborative team player supporting peer growth and team success.

```
This resume refocuses Hafsa Ali’s impressive data science background to emphasize multimodal pipeline engineering, ML prototyping, production deployment, cloud skills, and collaboration—all directly aligned with the Machine Learning Engineer role at RealAvatar. It maintains all factual elements while framing them for maximum relevance.

#### Dislplay the generated cover_letter.md file.

In [21]:
display(Markdown("./cover_letter.md"))

```markdown
Hafsa Ali  
San Francisco, CA  
Email: hafsa.ali@email.com | Phone: (123) 456-7890 | LinkedIn: https://linkedin.com/in/hafsa-ali  

[Date]

Hiring Manager  
AI Fund – RealAvatar Team  

Dear Hiring Manager,

I am writing to express my enthusiasm for the Machine Learning Engineer (LatAm) position at AI Fund’s RealAvatar initiative. With over seven years of hands-on experience building scalable machine learning models, designing low-latency multimodal AI data pipelines, and deploying production-ready solutions in AWS cloud environments, I am confident in my ability to contribute effectively to your cutting-edge AI avatar platform.

Throughout my career, I have specialized in creating and optimizing data pipelines and machine learning workflows that handle diverse data modalities—audio, video, and textual data—in real time. At AI Horizons Consulting, I led the design and implementation of Python-based ETL pipelines that integrated streaming and batch data sources for financial risk assessment. This experience aligns closely with your requirement to develop multimodal pipelines supporting verified AI avatars for real-time conversations. I have successfully engineered pipeline components that not only ensure low latency operation but also promote modularity and ease of extension for new AI use cases.

My proficiency with databases including SQL and PostgreSQL, alongside experience with vector stores such as Pinecone, enables me to efficiently collect, preprocess, and analyze heterogeneous datasets, supporting robust model training and inference. I have a demonstrated track record developing ML classification prototypes and optimizing large language model workflows, with continuous model evaluation and deployment managing model drift through automated monitoring systems — all critical to maintaining performant AI avatars.

In addition to technical capabilities, I bring a proactive and results-driven mindset cultivated in fast-paced startup and consulting environments. I thrive under minimal supervision, consistently delivering high-quality, scalable solutions through rigorous testing and automation frameworks. My communication skills have empowered me to bridge gaps between technical teams and product stakeholders, ensuring AI features meet strategic business requirements without sacrificing technical excellence.

My AWS Certified Machine Learning Specialty credential and Master’s degree in Data Science further reinforce my capacity to excel within AI Fund’s innovative and dynamic culture. I am genuinely excited about your mission to redefine human-computer interaction with verified, multimodal AI avatars and would be honored to contribute to this groundbreaking work alongside a visionary team led by Andrew Ng and Jeff Daniel.

Thank you for considering my application. I look forward to the opportunity to further discuss how my expertise aligns with RealAvatar’s ambitions and how I can help accelerate your AI avatar platform innovation.

Warm regards,  
Hafsa Ali  
```

#### Dislplay the generated interview_materials.md file.

In [ ]:
display(Markdown("./interview_materials.md"))